In [82]:
import pandas as pd

In [83]:
df = pd.read_csv('concepcion.csv', index_col = 0)
df.shape

(2500, 42)

In [84]:
df = df[['price', 'propertyType', 'size', 'exterior', 'rooms',
         'bathrooms', 'address', 'province', 'municipality',
         'latitude', 'longitude', 'status', 'newDevelopment', 'hasLift',
         'parkingSpace', 'priceByArea', 'detailedType', 'highlight']]

In [85]:
df

,price,propertyType,size,exterior,rooms,bathrooms,address,province,municipality,latitude,longitude,status,newDevelopment,hasLift,parkingSpace,priceByArea,detailedType,highlight
0,295000.0,flat,67.0,True,1,1,Calle de los Misterios,Madrid,Madrid,40.441226,-3.643123,good,False,False,NaN,4403.0,{'typology': 'flat'},NaN
1,710000.0,chalet,172.0,NaN,2,2,barrio Concepción,Madrid,Madrid,40.441675,-3.644040,renew,False,NaN,"{'hasParkingSpace': True, 'isParkingSpaceInclu...",4128.0,"{'typology': 'chalet', 'subTypology': 'semidet...",NaN
2,800000.0,flat,101.0,True,2,2,barrio Concepción,Madrid,Madrid,40.441166,-3.643758,newdevelopment,True,True,"{'hasParkingSpace': True, 'isParkingSpaceInclu...",7921.0,{'typology': 'flat'},NaN
3,995000.0,flat,190.0,True,3,2,barrio Concepción,Madrid,Madrid,40.441759,-3.644219,newdevelopment,True,True,"{'hasParkingSpace': True, 'isParkingSpaceInclu...",5237.0,{'typology': 'flat'},NaN
4,1020000.0,flat,177.0,True,3,2,barrio Concepción,Madrid,Madrid,40.440526,-3.643084,newdevelopment,True,True,"{'hasParkingSpace': True, 'isParkingSpaceInclu...",5763.0,{'typology': 'flat'},NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2495,699000.0,flat,65.0,False,2,2,Calle Conde De Peñalver,Madrid,Madrid,40.428102,-3.673420,renew,False,True,NaN,10754.0,{'typology': 'flat'},NaN
2496,980000.0,flat,150.0,True,4,2,barrio Lista,Madrid,Madrid,40.438901,-3.678145,good,False,True,NaN,6533.0,{'typology': 'flat'},NaN
2497,1870000.0,penthouse,201.0,True,3,2,Calle del Príncipe de Vergara,Madrid,Madrid,40.437791,-3.677965,renew,False,True,NaN,9303.0,"{'typology': 'flat', 'subTypology': 'penthouse'}",NaN
2498,401500.0,flat,98.0,True,4,1,Pasaje San Martin de Valdeiglesias,Madrid,Madrid,40.442448,-3.678345,renew,False,False,NaN,4097.0,{'typology': 'flat'},{'groupDescription': 'Destacado'}


In [86]:
# Exterior cleaned
df['exterior'] = df['exterior'].fillna(False)

In [87]:
# hasLift cleaned
df['hasLift'] = df['hasLift'].fillna(False)

In [88]:
# highlight cleaned
df['highlight'] = df['highlight'].fillna("{'groupDescription': None}")

In [89]:
# parkingSpace cleaned
df['parkingSpace'] = df['parkingSpace'].fillna("{'hasParkingSpace': False, 'isParkingSpaceIncludedInPrice': False, 'parkingSpacePrice': None}")

# Feature engineering

In [90]:
# parkingSpace included fixed
df['parkingSpace'] = df['parkingSpace'].apply(lambda x: "{'hasParkingSpace': True, 'isParkingSpaceIncludedInPrice': True, 'parkingSpacePrice': None}" if x == "{'hasParkingSpace': True, 'isParkingSpaceIncludedInPrice': True}" else x)

In [91]:
# unique parkingSpaces are now consistent
df['parkingSpace'].unique()

array(["{'hasParkingSpace': False, 'isParkingSpaceIncludedInPrice': False, 'parkingSpacePrice': None}",
       "{'hasParkingSpace': True, 'isParkingSpaceIncludedInPrice': True, 'parkingSpacePrice': None}",
       "{'hasParkingSpace': True, 'isParkingSpaceIncludedInPrice': False, 'parkingSpacePrice': 25000.0}",
       "{'hasParkingSpace': True, 'isParkingSpaceIncludedInPrice': False, 'parkingSpacePrice': 18000.0}",
       "{'hasParkingSpace': True, 'isParkingSpaceIncludedInPrice': False, 'parkingSpacePrice': 24000.0}",
       "{'hasParkingSpace': True, 'isParkingSpaceIncludedInPrice': False, 'parkingSpacePrice': 15000.0}",
       "{'hasParkingSpace': True, 'isParkingSpaceIncludedInPrice': False, 'parkingSpacePrice': 20000.0}",
       "{'hasParkingSpace': True, 'isParkingSpaceIncludedInPrice': False, 'parkingSpacePrice': 21000.0}",
       "{'hasParkingSpace': True, 'isParkingSpaceIncludedInPrice': False, 'parkingSpacePrice': 19000.0}",
       "{'hasParkingSpace': True, 'isParkingSpaceInc

In [92]:
# fixing detailedType
df['detailedType'] = df['detailedType'].apply(lambda x: "{'typology': 'flat', 'subTypology': 'standardFlat'}" if x == "{'typology': 'flat'}" else x)
df['detailedType'] = df['detailedType'].apply(lambda x: "{'typology': 'chalet', 'subTypology': 'standardChalet'}" if x == "{'typology': 'chalet'}" else x)

In [93]:
# Converting string of dicts --> dicts

from ast import literal_eval # Library for string evaluation

df['parkingSpace'] = df['parkingSpace'].apply(literal_eval)
df['highlight'] = df['highlight'].apply(literal_eval)
df['detailedType'] = df['detailedType'].apply(literal_eval)

In [94]:
df = pd.concat([df,
           pd.json_normalize(df['parkingSpace']),
           pd.json_normalize(df['highlight']),
           pd.json_normalize(df['detailedType'])],
          axis = 1)

In [95]:
df

,price,propertyType,size,exterior,rooms,bathrooms,address,province,municipality,latitude,...,parkingSpace,priceByArea,detailedType,highlight,hasParkingSpace,isParkingSpaceIncludedInPrice,parkingSpacePrice,groupDescription,typology,subTypology
0,295000.0,flat,67.0,True,1,1,Calle de los Misterios,Madrid,Madrid,40.441226,...,"{'hasParkingSpace': False, 'isParkingSpaceIncl...",4403.0,"{'typology': 'flat', 'subTypology': 'standardF...",{'groupDescription': None},False,False,NaN,None,flat,standardFlat
1,710000.0,chalet,172.0,False,2,2,barrio Concepción,Madrid,Madrid,40.441675,...,"{'hasParkingSpace': True, 'isParkingSpaceInclu...",4128.0,"{'typology': 'chalet', 'subTypology': 'semidet...",{'groupDescription': None},True,True,NaN,None,chalet,semidetachedHouse
2,800000.0,flat,101.0,True,2,2,barrio Concepción,Madrid,Madrid,40.441166,...,"{'hasParkingSpace': True, 'isParkingSpaceInclu...",7921.0,"{'typology': 'flat', 'subTypology': 'standardF...",{'groupDescription': None},True,True,NaN,None,flat,standardFlat
3,995000.0,flat,190.0,True,3,2,barrio Concepción,Madrid,Madrid,40.441759,...,"{'hasParkingSpace': True, 'isParkingSpaceInclu...",5237.0,"{'typology': 'flat', 'subTypology': 'standardF...",{'groupDescription': None},True,True,NaN,None,flat,standardFlat
4,1020000.0,flat,177.0,True,3,2,barrio Concepción,Madrid,Madrid,40.440526,...,"{'hasParkingSpace': True, 'isParkingSpaceInclu...",5763.0,"{'typology': 'flat', 'subTypology': 'standardF...",{'groupDescription': None},True,True,NaN,None,flat,standardFlat
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2495,699000.0,flat,65.0,False,2,2,Calle Conde De Peñalver,Madrid,Madrid,40.428102,...,"{'hasParkingSpace': False, 'isParkingSpaceIncl...",10754.0,"{'typology': 'flat', 'subTypology': 'standardF...",{'groupDescription': None},False,False,NaN,None,flat,standardFlat
2496,980000.0,flat,150.0,True,4,2,barrio Lista,Madrid,Madrid,40.438901,...,"{'hasParkingSpace': False, 'isParkingSpaceIncl...",6533.0,"{'typology': 'flat', 'subTypology': 'standardF...",{'groupDescription': None},False,False,NaN,None,flat,standardFlat
2497,1870000.0,penthouse,201.0,True,3,2,Calle del Príncipe de Vergara,Madrid,Madrid,40.437791,...,"{'hasParkingSpace': False, 'isParkingSpaceIncl...",9303.0,"{'typology': 'flat', 'subTypology': 'penthouse'}",{'groupDescription': None},False,False,NaN,None,flat,penthouse
2498,401500.0,flat,98.0,True,4,1,Pasaje San Martin de Valdeiglesias,Madrid,Madrid,40.442448,...,"{'hasParkingSpace': False, 'isParkingSpaceIncl...",4097.0,"{'typology': 'flat', 'subTypology': 'standardF...",{'groupDescription': 'Destacado'},False,False,NaN,Destacado,flat,standardFlat


In [96]:
del df['parkingSpace']
del df['highlight']
del df['detailedType']

In [103]:
df.columns

Index(['propertyType', 'size', 'exterior', 'rooms', 'bathrooms', 'address',
       'province', 'municipality', 'latitude', 'longitude', 'status',
       'newDevelopment', 'hasLift', 'priceByArea', 'hasParkingSpace',
       'isParkingSpaceIncludedInPrice', 'parkingSpacePrice',
       'groupDescription', 'typology', 'subTypology'],
      dtype='object')

In [101]:
df2.columns

Index(['price', 'propertyType', 'size', 'exterior', 'rooms', 'bathrooms',
       'address', 'province', 'municipality', 'latitude', 'longitude',
       'status', 'newDevelopment', 'hasLift', 'priceByArea', 'hasParkingSpace',
       'isParkingSpaceIncludedInPrice', 'parkingSpacePrice',
       'groupDescription', 'typology', 'subTypology'],
      dtype='object')

In [100]:
df2 = df

In [102]:
df = pd.read_csv('recleaned_df.csv', index_col = 0)

In [81]:
df.columns

Index(['propertyType', 'size', 'exterior', 'rooms', 'bathrooms', 'address',
       'province', 'municipality', 'latitude', 'longitude', 'status',
       'newDevelopment', 'hasLift', 'priceByArea', 'hasParkingSpace',
       'isParkingSpaceIncludedInPrice', 'parkingSpacePrice',
       'groupDescription', 'typology', 'subTypology'],
      dtype='object')

In [41]:
df.columns == df2.columns

ValueError: Lengths must match to compare

In [43]:
combined = pd.concat([df, df2], axis = 0)
combined.shape

(14810, 21)

In [44]:
combined = combined.drop_duplicates()
combined

,propertyType,size,exterior,rooms,bathrooms,address,province,municipality,latitude,longitude,...,newDevelopment,hasLift,priceByArea,hasParkingSpace,isParkingSpaceIncludedInPrice,parkingSpacePrice,groupDescription,typology,subTypology,price
335000.0,flat,136.0,True,3,2,SIGRID,Madrid,Rivas-Vaciamadrid,40.352160,-3.518342,...,False,True,2463.0,True,True,NaN,Destacado,flat,standardFlat,NaN
1650000.0,flat,179.0,True,2,3,Calle del Príncipe de Vergara,Madrid,Madrid,40.433593,-3.677788,...,False,True,9218.0,True,True,NaN,Destacado,flat,standardFlat,NaN
770000.0,flat,109.0,True,2,2,barrio Ibiza,Madrid,Madrid,40.416551,-3.672922,...,False,True,7064.0,False,False,NaN,Destacado,flat,standardFlat,NaN
175000.0,flat,49.0,True,1,1,"Calle de los Morales, 10",Madrid,Madrid,40.361994,-3.756360,...,False,True,3571.0,True,False,25000.0,Destacado,flat,standardFlat,NaN
280000.0,flat,121.0,True,4,2,Calle del Río Ulla,Madrid,Madrid,40.432982,-3.643371,...,False,False,2314.0,False,False,NaN,Destacado,flat,standardFlat,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2495.0,flat,65.0,False,2,2,Calle Conde De Peñalver,Madrid,Madrid,40.428102,-3.673420,...,False,True,10754.0,False,False,NaN,None,flat,standardFlat,699000.0
2496.0,flat,150.0,True,4,2,barrio Lista,Madrid,Madrid,40.438901,-3.678145,...,False,True,6533.0,False,False,NaN,None,flat,standardFlat,980000.0
2497.0,penthouse,201.0,True,3,2,Calle del Príncipe de Vergara,Madrid,Madrid,40.437791,-3.677965,...,False,True,9303.0,False,False,NaN,None,flat,penthouse,1870000.0
2498.0,flat,98.0,True,4,1,Pasaje San Martin de Valdeiglesias,Madrid,Madrid,40.442448,-3.678345,...,False,False,4097.0,False,False,NaN,Destacado,flat,standardFlat,401500.0


In [21]:
combined.to_csv('recleaned_df.csv', index = False)

# Recleaned

In [22]:
import pandas as pd

In [23]:
df

,price,propertyType,size,exterior,rooms,bathrooms,address,province,municipality,latitude,...,status,newDevelopment,hasLift,priceByArea,hasParkingSpace,isParkingSpaceIncludedInPrice,parkingSpacePrice,groupDescription,typology,subTypology
0,335000.0,flat,136.0,True,3,2,SIGRID,Madrid,Rivas-Vaciamadrid,40.352160,...,good,False,True,2463.0,True,True,NaN,Destacado,flat,standardFlat
1,1650000.0,flat,179.0,True,2,3,Calle del Príncipe de Vergara,Madrid,Madrid,40.433593,...,good,False,True,9218.0,True,True,NaN,Destacado,flat,standardFlat
2,770000.0,flat,109.0,True,2,2,barrio Ibiza,Madrid,Madrid,40.416551,...,good,False,True,7064.0,False,False,NaN,Destacado,flat,standardFlat
3,175000.0,flat,49.0,True,1,1,"Calle de los Morales, 10",Madrid,Madrid,40.361994,...,good,False,True,3571.0,True,False,25000.0,Destacado,flat,standardFlat
4,280000.0,flat,121.0,True,4,2,Calle del Río Ulla,Madrid,Madrid,40.432982,...,good,False,False,2314.0,False,False,NaN,Destacado,flat,standardFlat
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1344,160000.0,flat,90.0,True,3,1,Paseo de Vigo,Madrid,Fuenlabrada,40.294182,...,good,False,True,1778.0,True,True,NaN,NaN,flat,standardFlat
1345,220000.0,flat,101.0,True,3,2,Calle Valladolid,Madrid,Alcorcón,40.341681,...,good,False,True,2178.0,False,False,NaN,NaN,flat,standardFlat
1346,174900.0,flat,102.0,True,4,1,barrio Buena Vista,Madrid,Madrid,40.370577,...,good,False,False,1715.0,False,False,NaN,NaN,flat,standardFlat
1347,219000.0,flat,109.0,True,4,2,Calle del Grafito,Madrid,Madrid,40.347064,...,good,False,True,2009.0,True,True,NaN,Destacado,flat,standardFlat
